# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [8]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data is not None:
            # Initialize from a list of lists
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns.")
            self.data = data
            self.n = len(data)    
            self.m = len(data[0]) 
        else:
            # Initialize with dimensions n x m with all zeros
            if n is None or m is None:
                raise ValueError("Must provide dimensions n and m.")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]

    def __getitem__(self, index):
        if isinstance(index, tuple):
            # M[i, j] style indexing
            row, col = index
            return self.data[row][col]
        else:
            # M[i][j] style indexing
            return self.data[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            # M[i, j] style assignment
            row, col = index
            self.data[row][col] = value
        else:
            # M[i] style assignment
            self.data[index] = value

    def __eq__(self, other):
        # Check if two matrices are equal
        if self.n != other.n or self.m != other.m:
            return False
        return self.data == other.data

    def assign(self, other):
        if isinstance(other, Matrix):
            # Assign from another matrix
            if self.n != other.n or self.m != other.m:
                raise ValueError("Matrices must be the same size for assignment.")
            self.data = [row[:] for row in other.data]
        elif isinstance(other, list):
            # Assign from a list of lists
            if not all(len(row) == self.m for row in other):
                raise ValueError("The list of lists must match the matrix dimensions.")
            self.data = other
        else:
            raise TypeError("Assignment only supports matrices or list of lists.")

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

# Testing the class functionality
if __name__ == "__main__":
    # Test 1: Initialization with dimensions
    print("Test 1: Initialize matrix with dimensions 3x3")
    M1 = Matrix(3, 3)
    print(M1)

    # Test 2: Initialization with list of lists
    print("\nTest 2: Initialize matrix with a list of lists")
    M2 = Matrix(data=[[1, 2, 3], [4, 5, 6], [7, 8, 9]])
    print(M2)

    # Test 3: Invalid matrix initialization
    try:
        print("\nTest 3: Initialize matrix with incorrect list of lists")
        M3 = Matrix(data=[[1, 2], [3, 4, 5]])  # Unequal row lengths
    except ValueError as e:
        print(e)

    # Test 4: Indexing and assignment
    print("\nTest 4: Test indexing and assignment")
    print("Before assignment:")
    print(M1)
    M1[0, 0] = 10
    M1[2][2] = 99
    print("After assignment:")
    print(M1)

    # Test 5: Matrix assignment
    print("\nTest 5: Matrix assignment from another matrix")
    M4 = Matrix(3, 3)
    M4.assign(M2)
    print(M4)

    # Test 6: Matrix assignment with a list of lists
    print("\nTest 6: Matrix assignment with list of lists")
    M4.assign([[10, 11, 12], [13, 14, 15], [16, 17, 18]])
    print(M4)

    # Test 7: Invalid assignment (size mismatch)
    try:
        print("\nTest 7: Matrix assignment with size mismatch")
        M5 = Matrix(2, 2)
        M5.assign(M2)  
    except ValueError as e:
        print(e)


Test 1: Initialize matrix with dimensions 3x3
0 0 0
0 0 0
0 0 0

Test 2: Initialize matrix with a list of lists
1 2 3
4 5 6
7 8 9

Test 3: Initialize matrix with incorrect list of lists
All rows must have the same number of columns.

Test 4: Test indexing and assignment
Before assignment:
0 0 0
0 0 0
0 0 0
After assignment:
10 0 0
0 0 0
0 0 99

Test 5: Matrix assignment from another matrix
1 2 3
4 5 6
7 8 9

Test 6: Matrix assignment with list of lists
10 11 12
13 14 15
16 17 18

Test 7: Matrix assignment with size mismatch
Matrices must be the same size for assignment.


2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [9]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data is not None:
            # Here we are initializing from a list of lists
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns.")
            self.data = data
            self.n = len(data)    
            self.m = len(data[0]) 
        else:
            # Here we are initializing with dimensions n x m with all zeros
            if n is None or m is None:
                raise ValueError("Must provide dimensions n and m.")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]

    def shape(self):
        return (self.n, self.m)

    def transpose(self):
        transposed_data = [[self.data[j][i] for j in range(self.n)] for i in range(self.m)]
        return Matrix(data=transposed_data)

    def row(self, n):
        if n >= self.n:
            raise IndexError("Row index out of range.")
        return Matrix(data=[self.data[n]])

    def column(self, n):
        if n >= self.m:
            raise IndexError("Column index out of range.")
        col_data = [[self.data[i][n]] for i in range(self.n)]
        return Matrix(data=col_data)

    def to_list(self):
        return self.data

    def block(self, n_0, n_1, m_0, m_1):
        if not (0 <= n_0 < self.n and 0 < n_1 <= self.n and n_0 < n_1):
            raise IndexError("Row indices out of range.")
        if not (0 <= m_0 < self.m and 0 < m_1 <= self.m and m_0 < m_1):
            raise IndexError("Column indices out of range.")
        
        block_data = [row[m_0:m_1] for row in self.data[n_0:n_1]]
        return Matrix(data=block_data)

    def __getitem__(self, index):
        if isinstance(index, tuple):
            if isinstance(index[0], slice) or isinstance(index[1], slice):
                # Supporting slicing for rows and columns
                row_slice, col_slice = index
                row_range = range(*row_slice.indices(self.n))
                col_range = range(*col_slice.indices(self.m))
                sliced_data = [[self.data[r][c] for c in col_range] for r in row_range]
                return Matrix(data=sliced_data)
            else:
                # M[i, j] style indexing
                row, col = index
                return self.data[row][col]
        else:
            # M[i] style indexing
            return self.data[index]

    def __setitem__(self, index, value):
        if isinstance(index, tuple):
            # M[i, j] style assignment
            row, col = index
            self.data[row][col] = value
        else:
            # M[i] style assignment
            self.data[index] = value

    def __eq__(self, other):
        # Check if two matrices are equal
        if self.n != other.n or self.m != other.m:
            return False
        return self.data == other.data

    def assign(self, other):
        if isinstance(other, Matrix):
            # Assign from another matrix
            if self.n != other.n or self.m != other.m:
                raise ValueError("Matrices must be the same size for assignment.")
            self.data = [row[:] for row in other.data]
        elif isinstance(other, list):
            # Assign from a list of lists
            if not all(len(row) == self.m for row in other):
                raise ValueError("The list of lists must match the matrix dimensions.")
            self.data = other
        else:
            raise TypeError("Assignment only supports matrices or list of lists.")

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

# Testing the class functionality
if __name__ == "__main__":
    # Initialize a matrix for testing
    M = Matrix(data=[[1, 2, 3], [4, 5, 6], [7, 8, 9]])

    # Test shape
    print("Shape:", M.shape())  # Should be (3, 3)

    # Test transpose
    print("\nTranspose:")
    print(M.transpose())

    # Test row extraction
    print("\nRow 1:")
    print(M.row(1))  # Should return the second row as a matrix

    # Test column extraction
    print("\nColumn 1:")
    print(M.column(1))  # Should return the second column as a matrix

    # Test to_list
    print("\nMatrix as list of lists:")
    print(M.to_list())  # Should return the matrix as a list of lists

    # Test block extraction
    print("\nBlock (1, 3, 1, 3):")
    print(M.block(1, 3, 1, 3))  # Should return the bottom-right 2x2 block

    # Test slicing
    print("\nSlicing [1:3, 0:2]:")
    print(M[1:3, 0:2])  # Should return a submatrix


Shape: (3, 3)

Transpose:
1 4 7
2 5 8
3 6 9

Row 1:
4 5 6

Column 1:
2
5
8

Matrix as list of lists:
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

Block (1, 3, 1, 3):
5 6
8 9

Slicing [1:3, 0:2]:
4 5
7 8


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [11]:
def constant(n, m, c):
    """
    Returns a matrix of size n x m filled with floats of value c.
    """
    return [[float(c) for _ in range(m)] for _ in range(n)]

def zeros(n, m):
    """
    Returns a matrix of size n x m filled with floats of value 0.
    """
    return constant(n, m, 0)

def ones(n, m):
    """
    Returns a matrix of size n x m filled with floats of value 1.
    """
    return constant(n, m, 1)

def eye(n):
    """
    Returns an n x n identity matrix.
    """
    return [[1.0 if i == j else 0.0 for j in range(n)] for i in range(n)]

# Testing the functions
if __name__ == "__main__":
    print("Constant 3x3 matrix with value 5:")
    print(constant(3, 3, 5))

    print("\n3x4 zero matrix:")
    print(zeros(3, 4))

    print("\n2x2 ones matrix:")
    print(ones(2, 2))

    print("\n4x4 identity matrix:")
    print(eye(4))


Constant 3x3 matrix with value 5:
[[5.0, 5.0, 5.0], [5.0, 5.0, 5.0], [5.0, 5.0, 5.0]]

3x4 zero matrix:
[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]

2x2 ones matrix:
[[1.0, 1.0], [1.0, 1.0]]

4x4 identity matrix:
[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [12]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data is not None:
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns.")
            self.data = data
            self.n = len(data)    
            self.m = len(data[0]) 
        else:
            if n is None or m is None:
                raise ValueError("Must provide dimensions n and m.")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]

    def scalarmul(self, c):
        """
        Multiplies every element of the matrix by the scalar c.
        """
        return Matrix(data=[[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def add(self, N):
        """
        Adds two matrices. The dimensions must match.
        """
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must be the same size for addition.")
        return Matrix(data=[[self.data[i][j] + N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def sub(self, N):
        """
        Subtracts matrix N from the current matrix. The dimensions must match.
        """
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must be the same size for subtraction.")
        return Matrix(data=[[self.data[i][j] - N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def mat_mult(self, N):
        """
        Returns the matrix product of two matrices. The number of columns in the first matrix
        must equal the number of rows in the second matrix.
        """
        if self.m != N.n:
            raise ValueError("Matrices must be compatible for multiplication (m1 columns == m2 rows).")
        result = [[sum(self.data[i][k] * N.data[k][j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)]
        return Matrix(data=result)

    def element_mult(self, N):
        """
        Returns the element-wise product of two matrices. The dimensions must match.
        """
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must be the same size for element-wise multiplication.")
        return Matrix(data=[[self.data[i][j] * N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    def equals(self, N):
        """
        Checks if two matrices are equal.
        """
        if self.n != N.n or self.m != N.m:
            return False
        return self.data == N.data

    def __getitem__(self, index):
        if isinstance(index, tuple):
            row, col = index
            return self.data[row][col]
        else:
            return self.data[index]

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

# Testing the class functionality
if __name__ == "__main__":
    M = Matrix(data=[[1, 2], [3, 4]])
    N = Matrix(data=[[5, 6], [7, 8]])

    # Test scalar multiplication
    print("Scalar multiplication of M by 2:")
    print(M.scalarmul(2))

    # Test matrix addition
    print("\nAddition of M and N:")
    print(M.add(N))

    # Test matrix subtraction
    print("\nSubtraction of M and N:")
    print(M.sub(N))

    # Test matrix multiplication
    M3 = Matrix(data=[[1, 2, 3], [4, 5, 6]])
    N3 = Matrix(data=[[7, 8], [9, 10], [11, 12]])
    print("\nMatrix multiplication of M3 and N3:")
    print(M3.mat_mult(N3))

    # Test element-wise multiplication
    print("\nElement-wise multiplication of M and N:")
    print(M.element_mult(N))

    # Test equality
    print("\nEquality check between M and N:")
    print(M.equals(N))  # Should return False

    print("\nEquality check between M and itself:")
    print(M.equals(M))  # Should return True


Scalar multiplication of M by 2:
2 4
6 8

Addition of M and N:
6 8
10 12

Subtraction of M and N:
-4 -4
-4 -4

Matrix multiplication of M3 and N3:
58 64
139 154

Element-wise multiplication of M and N:
5 12
21 32

Equality check between M and N:
False

Equality check between M and itself:
True


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [5]:
class Matrix:
    def __init__(self, n=None, m=None, data=None):
        if data is not None:
            if not all(len(row) == len(data[0]) for row in data):
                raise ValueError("All rows must have the same number of columns.")
            self.data = data
            self.n = len(data)    # Number of rows
            self.m = len(data[0]) # Number of columns
        else:
            if n is None or m is None:
                raise ValueError("Must provide dimensions n and m.")
            self.n = n
            self.m = m
            self.data = [[0 for _ in range(m)] for _ in range(n)]

    # Scalar multiplication
    def __mul__(self, c):
        if isinstance(c, (int, float)):
            return self.scalarmul(c)
        elif isinstance(c, Matrix):
            return self.element_mult(c)  # Element-wise multiplication by default
        else:
            raise ValueError("Unsupported operation for type {}".format(type(c)))

    def __rmul__(self, c):
        # Handles cases where scalar is on the left, i.e., 2 * M
        return self.__mul__(c)

    # Matrix addition
    def __add__(self, N):
        return self.add(N)

    # Matrix subtraction
    def __sub__(self, N):
        return self.sub(N)

    # Matrix multiplication (dot product)
    def __matmul__(self, N):
        return self.mat_mult(N)

    # Matrix equality
    def __eq__(self, N):
        return self.equals(N)

    # Scalar multiplication function
    def scalarmul(self, c):
        return Matrix(data=[[c * self.data[i][j] for j in range(self.m)] for i in range(self.n)])

    # Matrix addition function
    def add(self, N):
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must be the same size for addition.")
        return Matrix(data=[[self.data[i][j] + N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    # Matrix subtraction function
    def sub(self, N):
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must be the same size for subtraction.")
        return Matrix(data=[[self.data[i][j] - N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    # Matrix multiplication (dot product)
    def mat_mult(self, N):
        if self.m != N.n:
            raise ValueError("Matrices must be compatible for multiplication (m1 columns == m2 rows).")
        result = [[sum(self.data[i][k] * N.data[k][j] for k in range(self.m)) for j in range(N.m)] for i in range(self.n)]
        return Matrix(data=result)

    # Element-wise multiplication
    def element_mult(self, N):
        if self.n != N.n or self.m != N.m:
            raise ValueError("Matrices must be the same size for element-wise multiplication.")
        return Matrix(data=[[self.data[i][j] * N.data[i][j] for j in range(self.m)] for i in range(self.n)])

    # Equality function
    def equals(self, N):
        if self.n != N.n or self.m != N.m:
            return False
        return self.data == N.data

    def __getitem__(self, index):
        if isinstance(index, tuple):
            row, col = index
            return self.data[row][col]
        else:
            return self.data[index]

    def __str__(self):
        return '\n'.join([' '.join(map(str, row)) for row in self.data])

# Testing the overloaded operators
if __name__ == "__main__":
    M = Matrix(data=[[1, 2], [3, 4]])
    N = Matrix(data=[[5, 6], [7, 8]])

    # Test scalar multiplication
    print("2 * M:")
    print(2 * M)

    print("\nM * 2:")
    print(M * 2)

    # Test matrix addition
    print("\nM + N:")
    print(M + N)

    # Test matrix subtraction
    print("\nM - N:")
    print(M - N)

    # Test element-wise multiplication
    print("\nM * N (element-wise multiplication):")
    print(M * N)

    # Test matrix equality
    print("\nM == N:")
    print(M == N)

    # Test matrix assignment
    print("\nM = N (assignment, just copying values):")
    M = N
    print(M)


2 * M:
2 4
6 8

M * 2:
2 4
6 8

M + N:
6 8
10 12

M - N:
-4 -4
-4 -4

M * N (element-wise multiplication):
5 12
21 32

M == N:
False

M = N (assignment, just copying values):
5 6
7 8


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [6]:
if __name__ == "__main__":
    # Define matrices A, B, C
    A = Matrix(data=[[1, 2], [3, 4]])
    B = Matrix(data=[[5, 6], [7, 8]])
    C = Matrix(data=[[9, 10], [11, 12]])
    I = Matrix(data=[[1, 0], [0, 1]])  # 2x2 Identity matrix

    # 1. (AB)C = A(BC) (Associativity)
    print("Associativity of matrix multiplication: (AB)C = A(BC)")
    AB = A @ B
    ABC1 = AB @ C   # (AB)C
    BC = B @ C
    ABC2 = A @ BC   # A(BC)
    print("(AB)C:")
    print(ABC1)
    print("A(BC):")
    print(ABC2)
    print("Are they equal? ", ABC1 == ABC2)

    # 2. A(B + C) = AB + AC (Distributivity)
    print("\nDistributivity of matrix multiplication: A(B + C) = AB + AC")
    B_plus_C = B + C
    A_B_plus_C = A @ B_plus_C   # A(B + C)
    AB = A @ B
    AC = A @ C
    AB_plus_AC = AB + AC   # AB + AC
    print("A(B + C):")
    print(A_B_plus_C)
    print("AB + AC:")
    print(AB_plus_AC)
    print("Are they equal? ", A_B_plus_C == AB_plus_AC)

    # 3. AB ≠ BA (Non-commutativity)
    print("\nNon-commutativity of matrix multiplication: AB ≠ BA")
    AB = A @ B
    BA = B @ A
    print("AB:")
    print(AB)
    print("BA:")
    print(BA)
    print("Are they equal? ", AB == BA)

    # 4. AI = A (Multiplication by identity matrix)
    print("\nMultiplication by identity matrix: AI = A")
    AI = A @ I
    print("AI:")
    print(AI)
    print("Is AI equal to A? ", AI == A)


Associativity of matrix multiplication: (AB)C = A(BC)
(AB)C:
413 454
937 1030
A(BC):
413 454
937 1030
Are they equal?  True

Distributivity of matrix multiplication: A(B + C) = AB + AC
A(B + C):
50 56
114 128
AB + AC:
50 56
114 128
Are they equal?  True

Non-commutativity of matrix multiplication: AB ≠ BA
AB:
19 22
43 50
BA:
23 34
31 46
Are they equal?  False

Multiplication by identity matrix: AI = A
AI:
1 2
3 4
Is AI equal to A?  True
